In [7]:

from tdt4290.parse_lidar import parse_lidar_data
import pandas as pd
import json
import librosa
from datetime import datetime
from typing import List
import pandas as pd
from tdt4290.types_common import LidarLabeledAudio
from tdt4290.label_audio_with_lidar import label_audio_with_lidar
from tdt4290.classify_lidar import classify_lidar
import librosa.display

# We'll need IPython.display's Audio widget
from IPython.display import Audio

In [8]:
# Check the sound file
# lidar = parse_lidar_data("./data/lidar_log_grilstad_22.09.22_10_00_13_00.txt")
# lidar_labeled_audio: List[LidarLabeledAudio] = []
# #print(lidar)
# sample_rate = 32000
# print("Some hello")
# for i, row in lidar.iterrows():
#     print("key", row)
#     enter = int(row["seconds_enter"] * sample_rate)
#     leave = int(row["seconds_leave"] * sample_rate)
#     print(enter, leave)
#     item: LidarLabeledAudio = {
#         "lidar_data": row,
#         "audio_sample": (enter, leave)
#     }
#     print("val", item)


In [9]:
sample_rate = 32000
audio_start = pd.to_datetime("2022-09-22 10:05:10.0000")
# first_car_enter_audio_time = pd.Timedelta("00:00:10.608")
audio_end = pd.Timedelta("00:00:10.608")

#step 1 - Read lidar log
lidar_data = parse_lidar_data("data/lidar_log_grilstad_22.09.22_10_00_13_00.txt")
#step 2 - Classify lidar
classified_lidar = classify_lidar(lidar_data)
#step 3 - link with audio
labeled = label_audio_with_lidar(
        classified_lidar,
        audio_start,
        audio_length=pd.Timedelta("02:32:58"),
    )
labeled

,y0,y1,height,width,front_area,datetime_enter,datetime_leave,duration,class_1,class_2,timedelta_start,timedelta_end,audio_start_s,audio_end_s
28,225,393,188,168,31584,2022-09-22 10:05:20.708,2022-09-22 10:05:21.255,0.547,medium,medium,0 days 00:00:10.708000,0 days 00:00:11.255000,9.9815,11.9815
30,147,312,185,165,30525,2022-09-22 10:05:58.322,2022-09-22 10:05:58.682,0.360,medium,medium,0 days 00:00:48.322000,0 days 00:00:48.682000,47.5020,49.5020
32,344,590,340,246,83640,2022-09-22 10:06:33.510,2022-09-22 10:06:35.804,2.294,large,large,0 days 00:01:23.510000,0 days 00:01:25.804000,83.6570,85.6570
34,362,539,145,177,25665,2022-09-22 10:06:53.551,2022-09-22 10:06:53.964,0.413,medium,medium_big,0 days 00:01:43.551000,0 days 00:01:43.964000,102.7575,104.7575
36,471,542,158,71,11218,2022-09-22 10:07:05.711,2022-09-22 10:07:05.831,0.120,small,small,0 days 00:01:55.711000,0 days 00:01:55.831000,114.7710,116.7710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,370,524,162,154,24948,2022-09-22 12:37:27.857,2022-09-22 12:37:28.163,0.306,medium,medium_small,0 days 02:32:17.857000,0 days 02:32:18.163000,9137.0100,9139.0100
1106,395,560,145,165,23925,2022-09-22 12:37:34.430,2022-09-22 12:37:34.790,0.360,medium,medium,0 days 02:32:24.430000,0 days 02:32:24.790000,9143.6100,9145.6100
1108,125,278,162,153,24786,2022-09-22 12:37:36.910,2022-09-22 12:37:37.217,0.307,medium,medium_small,0 days 02:32:26.910000,0 days 02:32:27.217000,9146.0635,9148.0635
1110,120,289,152,169,25688,2022-09-22 12:37:43.110,2022-09-22 12:37:43.483,0.373,medium,medium,0 days 02:32:33.110000,0 days 02:32:33.483000,9152.2965,9154.2965


In [10]:
# renames
def rename_lidar_columns(df: pd.DataFrame, filename: str):
    new_df = df.filter(["audio_start_s", "audio_end_s", "class_1"])
    new_df["filename"] = filename
    return new_df.rename(
        columns={
            "audio_start_s": "onset",
            "audio_end_s": "offset",
            "class_1": "event_label",
        }
    )
rename_lidar_columns(labeled, "lidar_data_with_audio_timestamps_22.csv")


,onset,offset,event_label,filename
28,9.9815,11.9815,medium,lidar_data_with_audio_timestamps_22.csv
30,47.5020,49.5020,medium,lidar_data_with_audio_timestamps_22.csv
32,83.6570,85.6570,large,lidar_data_with_audio_timestamps_22.csv
34,102.7575,104.7575,medium,lidar_data_with_audio_timestamps_22.csv
36,114.7710,116.7710,small,lidar_data_with_audio_timestamps_22.csv
...,...,...,...,...
1104,9137.0100,9139.0100,medium,lidar_data_with_audio_timestamps_22.csv
1106,9143.6100,9145.6100,medium,lidar_data_with_audio_timestamps_22.csv
1108,9146.0635,9148.0635,medium,lidar_data_with_audio_timestamps_22.csv
1110,9152.2965,9154.2965,medium,lidar_data_with_audio_timestamps_22.csv


In [11]:
# Load labeled data into file
labeled.to_csv("data/lidar_data_with_audio_timestamps_22.csv")

In [12]:
# same for data samples the 27. september part 1
audio_start_2 = pd.to_datetime("2022-09-27 12:20:15.774")
#step 1 - Read lidar log
lidar_data_2 = parse_lidar_data("data/lidar_log_grilstad_27.09.22_12_00_17_00.txt")
#step 2 - Classify lidar
classified_lidar_2 = classify_lidar(lidar_data_2)
#step 3 - link with audio
labeled_2 = label_audio_with_lidar(
        classified_lidar_2,
        audio_start_2,
        audio_length=pd.Timedelta("01:49:10"),
    )
labeled_2.to_csv("data/lidar_data_with_audio_timestamps_27_1.csv")
labeled_2

,y0,y1,height,width,front_area,datetime_enter,datetime_leave,duration,class_1,class_2,timedelta_start,timedelta_end,audio_start_s,audio_end_s
67,109,166,164,57,9348,2022-09-27 12:20:33.334,2022-09-27 12:20:33.480,0.146,small,small,0 days 00:00:17.560000,0 days 00:00:17.706000,16.6330,18.6330
68,112,273,188,161,30268,2022-09-27 12:20:44.521,2022-09-27 12:20:44.894,0.373,medium,medium,0 days 00:00:28.747000,0 days 00:00:29.120000,27.9335,29.9335
69,360,516,145,156,22620,2022-09-27 12:22:23.098,2022-09-27 12:22:23.524,0.426,medium,medium_small,0 days 00:02:07.324000,0 days 00:02:07.750000,126.5370,128.5370
70,346,512,157,166,26062,2022-09-27 12:22:36.592,2022-09-27 12:22:37.152,0.560,medium,medium,0 days 00:02:20.818000,0 days 00:02:21.378000,140.0980,142.0980
71,88,255,172,167,28724,2022-09-27 12:22:55.459,2022-09-27 12:22:55.819,0.360,medium,medium,0 days 00:02:39.685000,0 days 00:02:40.045000,158.8650,160.8650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,88,258,150,170,25500,2022-09-27 14:08:46.855,2022-09-27 14:08:47.241,0.386,medium,medium,0 days 01:48:31.081000,0 days 01:48:31.467000,6510.2740,6512.2740
503,390,559,150,169,25350,2022-09-27 14:08:56.096,2022-09-27 14:08:56.522,0.426,medium,medium,0 days 01:48:40.322000,0 days 01:48:40.748000,6519.5350,6521.5350
504,388,562,178,174,30972,2022-09-27 14:09:01.856,2022-09-27 14:09:02.268,0.412,medium,medium_big,0 days 01:48:46.082000,0 days 01:48:46.494000,6525.2880,6527.2880
505,67,230,167,163,27221,2022-09-27 14:09:03.189,2022-09-27 14:09:03.522,0.333,medium,medium,0 days 01:48:47.415000,0 days 01:48:47.748000,6526.5815,6528.5815


In [13]:

# same for data samples the 27. september part 2
audio_start_3 = pd.to_datetime("2022-09-27 14:11:59.999")
#step 1 - Read lidar log
lidar_data_3 = parse_lidar_data("data/lidar_log_grilstad_27.09.22_12_00_17_00.txt")
#step 2 - Classify lidar
classified_lidar_3 = classify_lidar(lidar_data_3)
#step 3 - link with audio
labeled_3 = label_audio_with_lidar(
        classified_lidar_3,
        audio_start_3,
        audio_length=pd.Timedelta("01:56:52"),
    )

labeled_3.to_csv("data/lidar_data_with_audio_timestamps_27_2.csv")
labeled_3

,y0,y1,height,width,front_area,datetime_enter,datetime_leave,duration,class_1,class_2,timedelta_start,timedelta_end,audio_start_s,audio_end_s
521,335,492,165,157,25905,2022-09-27 14:12:20.875,2022-09-27 14:12:21.155,0.280,medium,medium_small,0 days 00:00:20.876000,0 days 00:00:21.156000,20.0160,22.0160
522,358,526,189,168,31752,2022-09-27 14:12:25.676,2022-09-27 14:12:26.143,0.467,medium,medium,0 days 00:00:25.677000,0 days 00:00:26.144000,24.9105,26.9105
523,368,549,147,181,26607,2022-09-27 14:12:45.984,2022-09-27 14:12:46.356,0.372,medium,medium_big,0 days 00:00:45.985000,0 days 00:00:46.357000,45.1710,47.1710
524,361,607,339,246,83394,2022-09-27 14:13:45.492,2022-09-27 14:13:47.972,2.480,large,large,0 days 00:01:45.493000,0 days 00:01:47.973000,105.7330,107.7330
525,57,223,173,166,28718,2022-09-27 14:13:47.866,2022-09-27 14:13:48.158,0.292,medium,medium,0 days 00:01:47.867000,0 days 00:01:48.159000,107.0130,109.0130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,375,549,154,174,26796,2022-09-27 16:08:28.094,2022-09-27 16:08:28.453,0.359,medium,medium_big,0 days 01:56:28.095000,0 days 01:56:28.454000,6987.2745,6989.2745
1341,66,236,164,170,27880,2022-09-27 16:08:31.627,2022-09-27 16:08:31.960,0.333,medium,medium,0 days 01:56:31.628000,0 days 01:56:31.961000,6990.7945,6992.7945
1342,386,544,151,158,23858,2022-09-27 16:08:32.654,2022-09-27 16:08:33.068,0.414,medium,medium_small,0 days 01:56:32.655000,0 days 01:56:33.069000,6991.8620,6993.8620
1343,411,573,161,162,26082,2022-09-27 16:08:35.254,2022-09-27 16:08:35.694,0.440,medium,medium,0 days 01:56:35.255000,0 days 01:56:35.695000,6994.4750,6996.4750


In [14]:

# same for data samples the 21. october 
audio_start_4 = pd.to_datetime("2022-10-21 09:18:36")
#step 1 - Read lidar log
lidar_data_4 = parse_lidar_data("data/lidar_log_grilstad_21.10.22_09_00_14_00.txt")
#step 2 - Classify lidar
classified_lidar_4 = classify_lidar(lidar_data_4)
#step 3 - link with audio
labeled_4 = label_audio_with_lidar(
        classified_lidar_4,
        audio_start_4,
        audio_length=pd.Timedelta("02:19:17"),
    )

labeled_4.to_csv("data/lidar_data_with_audio_timestamps_oct_21_1.csv")
labeled_4

,y0,y1,height,width,front_area,datetime_enter,datetime_leave,duration,class_1,class_2,timedelta_start,timedelta_end,audio_start_s,audio_end_s
54,63,305,339,242,82038,2022-10-21 09:18:44.443,2022-10-21 09:18:46.683,2.240,large,large,0 days 00:00:08.443000,0 days 00:00:10.683000,8.5630,10.5630
55,97,263,158,166,26228,2022-10-21 09:19:13.164,2022-10-21 09:19:13.536,0.372,medium,medium,0 days 00:00:37.164000,0 days 00:00:37.536000,36.3500,38.3500
56,122,298,163,176,28688,2022-10-21 09:19:18.657,2022-10-21 09:19:19.070,0.413,medium,medium_big,0 days 00:00:42.657000,0 days 00:00:43.070000,41.8635,43.8635
57,394,561,164,167,27388,2022-10-21 09:19:39.950,2022-10-21 09:19:40.365,0.415,medium,medium,0 days 00:01:03.950000,0 days 00:01:04.365000,63.1575,65.1575
58,340,513,196,173,33908,2022-10-21 09:20:36.513,2022-10-21 09:20:37.087,0.574,medium,medium_big,0 days 00:02:00.513000,0 days 00:02:01.087000,119.8000,121.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,394,565,160,171,27360,2022-10-21 11:36:51.939,2022-10-21 11:36:52.258,0.319,medium,medium_big,0 days 02:18:15.939000,0 days 02:18:16.258000,8295.0985,8297.0985
582,102,274,160,172,27520,2022-10-21 11:36:56.019,2022-10-21 11:36:56.406,0.387,medium,medium_big,0 days 02:18:20.019000,0 days 02:18:20.406000,8299.2125,8301.2125
583,356,527,143,171,24453,2022-10-21 11:37:01.632,2022-10-21 11:37:01.992,0.360,medium,medium_big,0 days 02:18:25.632000,0 days 02:18:25.992000,8304.8120,8306.8120
584,394,564,162,170,27540,2022-10-21 11:37:09.046,2022-10-21 11:37:09.433,0.387,medium,medium,0 days 02:18:33.046000,0 days 02:18:33.433000,8312.2395,8314.2395


In [15]:


# same for data samples the 21. october 
audio_start_5 = pd.to_datetime("2022-10-21 11:40:11")
#step 1 - Read lidar log
lidar_data_5 = parse_lidar_data("data/lidar_log_grilstad_21.10.22_09_00_14_00.txt")
#step 2 - Classify lidar
classified_lidar_5 = classify_lidar(lidar_data_5)
#step 3 - link with audio
labeled_5 = label_audio_with_lidar(
        classified_lidar_5,
        audio_start_5,
        audio_length=pd.Timedelta("01:31:24"),
    )

labeled_5.to_csv("data/lidar_data_with_audio_timestamps_oct_21_2.csv")
labeled_5

,y0,y1,height,width,front_area,datetime_enter,datetime_leave,duration,class_1,class_2,timedelta_start,timedelta_end,audio_start_s,audio_end_s
598,144,303,189,159,30051,2022-10-21 11:40:47.014,2022-10-21 11:40:47.440,0.426,medium,medium_small,0 days 00:00:36.014000,0 days 00:00:36.440000,35.2270,37.2270
599,108,283,151,175,26425,2022-10-21 11:41:17.508,2022-10-21 11:41:17.881,0.373,medium,medium_big,0 days 00:01:06.508000,0 days 00:01:06.881000,65.6945,67.6945
600,124,283,143,159,22737,2022-10-21 11:41:30.388,2022-10-21 11:41:30.696,0.308,medium,medium_small,0 days 00:01:19.388000,0 days 00:01:19.696000,78.5420,80.5420
601,357,528,141,171,24111,2022-10-21 11:41:53.696,2022-10-21 11:41:54.149,0.453,medium,medium_big,0 days 00:01:42.696000,0 days 00:01:43.149000,101.9225,103.9225
602,102,279,166,177,29382,2022-10-21 11:42:15.577,2022-10-21 11:42:15.923,0.346,medium,medium_big,0 days 00:02:04.577000,0 days 00:02:04.923000,123.7500,125.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,110,264,168,154,25872,2022-10-21 13:10:24.177,2022-10-21 13:10:24.765,0.588,medium,medium_small,0 days 01:30:13.177000,0 days 01:30:13.765000,5412.4710,5414.4710
977,381,546,162,165,26730,2022-10-21 13:10:25.351,2022-10-21 13:10:25.697,0.346,medium,medium,0 days 01:30:14.351000,0 days 01:30:14.697000,5413.5240,5415.5240
978,367,538,190,171,32490,2022-10-21 13:10:46.525,2022-10-21 13:10:46.978,0.453,medium,medium_big,0 days 01:30:35.525000,0 days 01:30:35.978000,5434.7515,5436.7515
979,363,526,190,163,30970,2022-10-21 13:11:13.472,2022-10-21 13:11:13.886,0.414,medium,medium,0 days 01:31:02.472000,0 days 01:31:02.886000,5461.6790,5463.6790


In [16]:
sound, sr = librosa.load(".data/audio_21102022_1.WAV", sr=sample_rate, mono=True)
print("Sound length", len(sound))

Sound length 267115008


In [19]:

sound2, sr2 = librosa.load(".data/audio_21102022_2.WAV", sr=sample_rate, mono=True)
print("Sound length", len(sound2))

Sound length 175508352


In [17]:
32000 * 60 * 60 * 2.5

288000000.0

In [40]:
start, end = int(labeled_5["audio_start_s"][600] * sr), int(labeled_5["audio_end_s"][600] * sr)
print(start, end)
print(labeled_5.loc[600])
# diff = end - start
# new_start = int((start + diff/2) - 1 * 32000)
# new_end = int((end - diff/2) + 1 * 32000)
# part = sound[new_start:new_end]
# start, end = 8482912, 8569056
part = sound2[start:end]
Audio(data=part, rate=sample_rate)

2513344 2577344
y0                                        124
y1                                        283
height                                    143
width                                     159
front_area                              22737
datetime_enter     2022-10-21 11:41:30.388000
datetime_leave     2022-10-21 11:41:30.696000
duration                                0.308
class_1                                medium
class_2                          medium_small
timedelta_start        0 days 00:01:19.388000
timedelta_end          0 days 00:01:19.696000
audio_start_s                          78.542
audio_end_s                            80.542
Name: 600, dtype: object
